In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
import pickle
import warnings
warnings.filterwarnings("ignore")

In [2]:
def selectkbest(indep_X,dep_Y,n):
        test = SelectKBest(score_func=f_regression, k=n)
        fit1= test.fit(indep_X,dep_Y)
        selectk_features = fit1.transform(indep_X)
        selectk_features = fit1.transform(indep_X)
        selected_indices = fit1.get_support(indices=True) 
        selected_features_names = indep_X.columns[selected_indices]
        return selectk_features,selected_features_names.to_list()

In [3]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test,sc

In [4]:
#Evaluation Metrics for Regression Models
def R2_prediction(regressor,x_test):
    y_pred = regressor.predict(x_test)
    
    from sklearn.metrics import r2_score
    R2_score = r2_score(y_test,y_pred)
    return regressor,R2_score,x_test,y_test

In [5]:
def svm_linear(x_train,y_train,x_test):
    from sklearn.svm import SVR
    param_grid={'kernel':['rbf'],'C':[100],'gamma':['scale']}
    grid=GridSearchCV(SVR(), param_grid, refit=True, verbose=1,n_jobs=1)
    grid.fit(x_train,y_train)
    R2_score=R2_prediction(grid,x_test)
    print ("The Rsquare value for the the best paramater {}:".format(grid.best_params_))
    return grid,R2_score,x_test,y_test

In [6]:
def selectk_regression(r2rf):
    dataframe = pd.DataFrame(index=['R2 Score'], columns = ['SVM_L'])
    for number,index in enumerate(dataframe.index):
        dataframe['SVM_L'][index] = r2svml[number] 
    return dataframe

In [7]:
data=pd.read_csv("clean_data.csv")
data

,Shipment_ID,Distance_km,Weight_kg,Mode_of_Transport,Origin,Destination,Fuel_Price_per_Liter,Demand_Fluctuation,Traffic_Conditions,Warehouse_Processing_Time,Customs_Clearance_Delay,Packaging_Cost,Freight_Insurance_Cost,Freight_Cost_USD
0,SHIP00001,2588.5,1341.000000,Rail,Kolkata,London,1.89,1.042210,Medium,8,2,326.35,969.69,818.740237
1,SHIP00002,3822.0,3538.000000,Sea,Kolkata,Los Angeles,0.75,1.075883,High,8,3,293.82,649.12,676.720000
2,SHIP00003,3142.0,4777.000000,Road,Chennai,London,1.99,1.170771,Low,3,2,52.03,150.30,1288.520000
3,SHIP00004,516.0,3704.000000,Air,Chennai,Dubai,1.58,1.129244,Low,8,3,421.87,181.12,1267.500000
4,SHIP00005,4476.0,578.000000,Sea,Mumbai,Dubai,1.26,0.998045,High,5,4,428.14,615.63,463.680000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,SHIP09996,284.0,633.000000,Air,Bangalore,Los Angeles,0.52,0.977983,Medium,2,4,309.59,739.37,91.620000
9996,SHIP09997,2588.5,2859.000000,Rail,Kolkata,Los Angeles,1.39,1.091401,Medium,8,1,192.02,386.04,472.500000
9997,SHIP09998,1958.0,705.000000,Air,Chennai,New York,1.26,0.971334,Medium,8,1,125.19,323.12,599.220000
9998,SHIP09999,1467.0,4931.000000,Rail,Kolkata,Singapore,1.56,0.973844,High,1,1,107.21,489.50,716.910000


In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data["Origin"] = encoder.fit_transform(data["Origin"])
data["Destination"] = encoder.fit_transform(data["Destination"])
data["Mode_of_Transport"] = encoder.fit_transform(data["Mode_of_Transport"])
data["Traffic_Conditions"] = encoder.fit_transform(data["Traffic_Conditions"])

In [9]:
data=data.drop('Shipment_ID',axis=1)

In [10]:
data=pd.get_dummies(data,drop_first=True)
data

,Distance_km,Weight_kg,Mode_of_Transport,Origin,Destination,Fuel_Price_per_Liter,Demand_Fluctuation,Traffic_Conditions,Warehouse_Processing_Time,Customs_Clearance_Delay,Packaging_Cost,Freight_Insurance_Cost,Freight_Cost_USD
0,2588.5,1341.000000,1,3,1,1.89,1.042210,2,8,2,326.35,969.69,818.740237
1,3822.0,3538.000000,3,3,2,0.75,1.075883,0,8,3,293.82,649.12,676.720000
2,3142.0,4777.000000,2,1,1,1.99,1.170771,1,3,2,52.03,150.30,1288.520000
3,516.0,3704.000000,0,1,0,1.58,1.129244,1,8,3,421.87,181.12,1267.500000
4,4476.0,578.000000,3,4,0,1.26,0.998045,0,5,4,428.14,615.63,463.680000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,284.0,633.000000,0,0,2,0.52,0.977983,2,2,4,309.59,739.37,91.620000
9996,2588.5,2859.000000,1,3,2,1.39,1.091401,2,8,1,192.02,386.04,472.500000
9997,1958.0,705.000000,0,1,3,1.26,0.971334,2,8,1,125.19,323.12,599.220000
9998,1467.0,4931.000000,1,3,4,1.56,0.973844,0,1,1,107.21,489.50,716.910000


In [11]:
indep_X=data.drop('Freight_Cost_USD',axis=1)
dep_Y=data['Freight_Cost_USD']

In [12]:
kbest,selected_features=selectkbest(indep_X,dep_Y,5)

In [13]:
selected_features

['Distance_km',
 'Weight_kg',
 'Mode_of_Transport',
 'Fuel_Price_per_Liter',
 'Demand_Fluctuation']

In [14]:
kbest

array([[2.58850000e+03, 1.34100000e+03, 1.00000000e+00, 1.89000000e+00,
        1.04220962e+00],
       [3.82200000e+03, 3.53800000e+03, 3.00000000e+00, 7.50000000e-01,
        1.07588253e+00],
       [3.14200000e+03, 4.77700000e+03, 2.00000000e+00, 1.99000000e+00,
        1.17077117e+00],
       ...,
       [1.95800000e+03, 7.05000000e+02, 0.00000000e+00, 1.26000000e+00,
        9.71333983e-01],
       [1.46700000e+03, 4.93100000e+03, 1.00000000e+00, 1.56000000e+00,
        9.73844311e-01],
       [4.96900000e+03, 2.49503989e+03, 2.00000000e+00, 1.20000000e+00,
        1.05822176e+00]])

In [15]:
r2svml = []

In [16]:
x_train,x_test,y_train,y_test,sc = split_scalar(kbest,dep_Y)

In [17]:
regressor,R2_score,x_test,y_test = svm_linear(x_train,y_train,x_test)
r2svml.append(R2_score[1])

result = selectk_regression(r2svml)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
The Rsquare value for the the best paramater {'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}:


In [18]:
result

,SVM_L
R2 Score,0.866211


In [19]:
# Create file for the model

filename = 'Best_Model.sav'

In [20]:
import pickle as pkl
pickle.dump(regressor,open(filename,'wb'))

In [21]:
# Create file for the preprocessing step

filenamesc = 'sc.pkl'

In [23]:
pickle.dump(sc,open(filenamesc,'wb')) 

In [24]:
selected_features

['Distance_km',
 'Weight_kg',
 'Mode_of_Transport',
 'Fuel_Price_per_Liter',
 'Demand_Fluctuation']

In [25]:
user_input = sc.transform([[2341,150, 1, 1.89,0]])
user_input

array([[-0.16351328, -1.68717418, -0.44420492,  1.50854754, -8.74237919]])

In [27]:
loaded_model = pickle.load(open('Best_Model.sav','rb'))
Prediction = loaded_model.predict(user_input)

In [28]:
Prediction

array([850.05306287])